## Създаване на примерни данни с python и faker


`Faker` е пакет за програмния език python, който представлява [генератор за псевдослучайни данни](https://bg.wikipedia.org/wiki/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80_%D0%BD%D0%B0_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D0%B8_%D1%87%D0%B8%D1%81%D0%BB%D0%B0). С множеството си предефинирани варианти за създаване на неистински данни и възможностите за конфигуриране, `Faker` улеснява работата на програмисти, тестъри, разработчици на бази-данни, както и на всички, които имат нужда от реалистично изглеждащи генерирани данни за работата си.

Повече информация на тези връзки:
- https://pypi.org/project/Faker/ (PyPI страница)
- https://github.com/joke2k/faker (Github страница)
- https://faker.readthedocs.io/en/master/ (Документация на Faker)

### Работа с `Faker` за генериране на псевдослучайни данни


Започваме с внасяне на `faker` в работната среда. Ще ни трябват още и модулите `datetime` и `csv` от стандартната библиотека на python:

In [8]:
import csv
import datetime
from faker import Faker

Инициализираме `Faker` обект, като подаваме начален ключ (в случая числото 0). Този ключ дава възможност на по-късен етап да бъде генерирана отново същата последователност от примерни данни. 

In [9]:
fake = Faker()
Faker.seed(0)

За описание на логиката на генериране на самите данни, Faker предоставя набор от модули, наречени доставчици (providers). Документацията описва два основни раздела доставчици -- standard providers (стандартни доставчици), написани от програмистите на Faker, и community providers, написани от външни разработчици. 


Нека вземем за пример `faker.providers.profile.Provider`. Това е генератор, който представлява колекция от функции за генериране на неистински идентичности. В базовата си форма той генерира речник с пълен набор от полета за фиктивен профил:

In [21]:
fake.profile()

{'job': 'Engineer, electronics',
 'company': 'Thornton-Lee',
 'ssn': '415-08-1039',
 'residence': '4273 Holt Motorway\nRodneyton, DC 64140',
 'current_location': (Decimal('-48.736401'), Decimal('-9.872198')),
 'blood_group': 'A+',
 'website': ['http://mckay.com/',
  'http://www.martin.com/',
  'http://daniels.org/',
  'https://hughes.com/'],
 'username': 'douglas91',
 'name': 'Amy Farrell',
 'sex': 'F',
 'address': '843 Stephanie Fort\nDanastad, CT 23614',
 'mail': 'alexandergreen@hotmail.com',
 'birthdate': datetime.date(1939, 10, 29)}

Можем да изберем кои полета да бъдат генерирани като подадем като параметър лист с техните имена. Като втори параметър можем да подадем пол:

In [23]:
fake.profile(["name", "job", "username"], "F")

{'job': 'Biomedical scientist',
 'username': 'susanhaynes',
 'name': 'Taylor Small'}

### Локализации

Конструкторът на `faker.Faker` може също така да приеме една (под формата на символен низ) или много (под формата на лист от стрингове) настройки за локализация. Това позволява генерирането на данни, релевантни за подадените региони. Списък на локализираните провайдъри е наличен в [документацията на Faker](https://faker.readthedocs.io/en/master/locales.html).

In [35]:
bg_fake = Faker('bg_BG')
for _ in range(6):
    print(f"{bg_fake.name()}, тел. {bg_fake.phone_number()}")

Акашия Братухчев, тел. +359(0)478501033
Г-ца Елина Вампирска, тел. +359(0)764 185781
Г-ца Елиезер Тухчиев, тел. +359220080276
Г-жа Кибер Топков, тел. 0874 203-609
Др. Лихия ЕвроповКирилов, тел. (0952) 706 502
Гримяна Кривошапкова, тел. +359421319291


### Разпределение на вероятностите


For probability distribution:

fake.random_elements(elements=OrderedDict([("a", 0.45), ("b", 0.35), ("c", 0.15), ("d", 0.05), ]), length=20, unique=False)

### Уникални стойности

Чрез достъпване на свойството `unique` на генератора, гарантираме че създадените стойности ще бъдат уникални. 

In [38]:
print([fake.unique.language_name() for _ in range(30)])

['Assamese', 'Javanese', 'Lao', 'Igbo', 'Czech', 'Japanese', 'Tajik', 'Romanian', 'Kuanyama', 'Chichewa', 'Herero', 'Latvian', 'Venda', 'Swedish', 'Dzongkha', 'Mongolian', 'Ndonga', 'Aymara', 'Interlingue', 'Limburgan', 'Persian', 'Sichuan Yi', 'Occitan', 'Hungarian', 'Swati', 'Bihari languages', 'Norwegian', 'Kongo', 'Turkmen', 'Maori']


### Пример -- Създаване на csv файл с фиктивни данни за клиенти

За да демонстрираме възможностите на `Faker`, нека да генерираме един текстов файл с делимитирани стойности. 

In [30]:
header = ["name", "sex", "birthdate", "username", "address", "SSN", "mail"]
rows_number = 100
output_filename = "faker_csv_write.csv"

In [31]:
%%time
with open(output_filename, "wt") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=header, delimiter=";")
    writer.writeheader()
    for i in range(rows_number):
        writer.writerow({
            "name": fake.name(),
            "sex": fake.random_element(elements=('M', 'F')),
            "birthdate": fake.date(pattern="%d-%m-%Y", end_datetime=datetime.date(2002, 1, 1)),
            "username": fake.profile(fields=["username"]).get("username"),
            "address": fake.address().replace("\n", " "),
            "SSN": fake.ssn(),
            "mail": fake.email(),
        })
    

CPU times: user 163 ms, sys: 22.2 ms, total: 186 ms
Wall time: 184 ms


Нека видим как изглеждат първите няколко реда на новосъздадения файл:

In [32]:
!head over_a_hundred_thousand.csv -n 5